In [9]:
!pip install --quiet google-play-scraper

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

# Core library
import pandas as pd

# Web scraping
from google_play_scraper import (
    Sort,
    reviews
)

pd.set_option('display.max_colwidth', None)

print('Ready!')

Ready!


In [ ]:
result, continuation_token = reviews(
    'com.tokopedia.tkpd',
    lang='id',             # Indonesian Language
    country='id',          # Region
    sort=Sort.NEWEST,      # Sort by the newest
    count=500_000,         # The amount of review to scrap
    filter_score_with=None
)

print('Data scraped successfully!')

Data scraped successfully!


In [ ]:
df_scrape = pd.DataFrame(result)

# Choose only the necessary column
df_raw = df_scrape[['content', 'score', 'at']]
df_raw.columns = ['raw_text', 'rating', 'date']
df_raw.head()

,raw_text,rating,date
0,Aplikasi bagun untuk belanja,4,2025-11-26 16:54:58
1,sudah mantap,5,2025-11-26 16:52:43
2,"tokopedia sekarang jadi ribet tidak seperti dulu lagi kalo mau return barang yang tidak sesuai harus nunggu waktu terlalu lama jadi males belanja lagi di tokopedia saya auto unistal, tidak seperti shopee yang mudah dan enak dan sekarang belaja terus di shopee",1,2025-11-26 16:30:31
3,"kasih bintang 1 ,karena ngisi kouta saja lama prosesnya",1,2025-11-26 15:05:33
4,"di janjikan dapet kompensasi atas keterlambatan pengiriman sameday yg gak sesuai estimasi, sampai sekarang udh 8 hari kerja blm dapet jga gimana sih Tokopedia",1,2025-11-26 15:03:32


In [ ]:
df_raw.to_csv('../data/raw/review.csv')
print('Scraped data successfully saved to "../data/raw/review.csv"')

Scraped data successfully saved to "../data/raw/review.csv"
